# Simple structured data demo

This is a very basic demo of how to log structured datasets for classification tasks with XGBoost.

In [1]:
import os

os.environ['GALILEO_CONSOLE_URL']="http://localhost:8088"
os.environ["GALILEO_USERNAME"]="user@example.com"
os.environ["GALILEO_PASSWORD"]="Th3secret_"

# os.environ['GALILEO_CONSOLE_URL']="https://console.dev.rungalileo.io"
# os.environ["GALILEO_USERNAME"]="galileo@rungalileo.io"
# os.environ["GALILEO_PASSWORD"]="A11a1una!"

import dataquality as dq
dq.configure()

dq.init("structured_classification", "structured-elliott", "fine-wine")

/Users/elliottchartock/Code/dataquality/.venv/lib/python3.9/site-packages/dataquality/__init__.py:75: GalileoWarning: configure is deprecated, use dq.set_console_url and dq.login
  warnings.warn(


📡 http://localhost:8088
🔭 Logging you into Galileo

🚀 You're logged in to Galileo as user@example.com!
✨ Initializing existing public project 'structured-elliott'
🏃‍♂️ Fetching existing run 'fine-wine'
🛰 Connected to existing project 'structured-elliott', and existing run 'fine-wine'.


/Users/elliottchartock/Code/dataquality/.venv/lib/python3.9/site-packages/dataquality/core/init.py:159: UserWarning: Run: structured-elliott/fine-wine already exists! The existing run will get overwritten on call to finish()!
  warnings.warn(


## 1. Load data

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.datasets import load_wine

X, y = load_wine(as_frame=True, return_X_y=True)

# When exporting to hdf5 you can't have col names containing forward slash
X.rename(lambda x: x.replace("/", "|"), axis="columns", inplace=True)


def understand_dataset(dataset) -> None:
    for elem in dir(dataset):
        print(elem, type(getattr(dataset, elem)))

    print(dataset.shape, y.shape)
    
#understand_dataset(wine)

## 2. Create and fit model on training data

In [3]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

## 3. Set labels for run

In [4]:
target_names = load_wine().target_names
dq.set_labels_for_run(target_names)

## 4. Log data!

In [5]:
dq.log_xgboost(
    model=xgb_model,
    X=X,
    y=y,
    split="training"
)

In [6]:
dq.log_xgboost(
    model=xgb_model,
    X=X,
    y=y,
    split="test"
)

## 5. Call finish to start processing

In [7]:
dq.finish()

☁️ Uploading Data


Uploading data to Galileo:   0%|          | 0.00/26.2k [00:00<?, ?B/s]

Uploading data to Galileo:   0%|          | 0.00/49.6k [00:00<?, ?B/s]

Uploading data to Galileo:   0%|          | 0.00/26.9k [00:00<?, ?B/s]

Uploading data to Galileo:   0%|          | 0.00/49.6k [00:00<?, ?B/s]

Job default successfully submitted. Results will be available soon at http://127.0.0.1:3000/insights?projectId=49128c2e-7379-4292-af6c-661b9b515d3f&runId=0db99692-7268-4f48-a10b-f9dcf10a8a15&split=training&metric=f1&depHigh=1&depLow=0&taskType=4
Waiting for job...


GalileoException: It seems your run failed with error
Failed background job: No data in minio for object 49128c2e-7379-4292-af6c-661b9b515d3f/0db99692-7268-4f48-a10b-f9dcf10a8a15/training/data/emb/emb.hdf5

Traceback (most recent call last):
  File "/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/rungalileo/objectstore.py", line 301, in download_object
    self.minio_client.fget_object(
  File "/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/minio/api.py", line 1042, in fget_object
    stat = self.stat_object(
  File "/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/minio/api.py", line 1868, in stat_object
    response = self._execute(
  File "/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/minio/api.py", line 403, in _execute
    return self._url_open(
  File "/Users/elliottchartock/Code/runners/.venv/lib/python3.9/site-packages/minio/api.py", line 386, in _url_open
    raise response_error
minio.er

## Feature Pair Accuracy Matrix

In [ ]:
# TODO

## Feature Importance

In [ ]:
xgb_model.feature_importances_

## Feature Correlation Matrix

In [ ]:
import vaex

df = vaex.from_pandas(X)
df.correlation(x=df.get_column_names())